In [1]:
import mne
import os
import numpy as np
import pandas as pd
import pywt
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay

# Directory paths
edf_dir = r'physionet.org/files/hmc-sleep-staging/1.0.0/recordings'  # Replace with the actual path to your EDF files
annotation_dir = r'physionet.org/files/hmc-sleep-staging/1.0.0/recordings'  # Replace with the actual path to your annotation files

# Initialize lists to store features and labels
all_wavelet_features = []
all_labels = []

# Define the wavelet transform function
def compute_wavelet_features(data, wavelet='db4', level=4):
    coeffs = pywt.wavedec(data, wavelet, level=level)
    # Extract statistics from each wavelet level (mean, std deviation)
    features = [np.mean(c) for c in coeffs] + [np.std(c) for c in coeffs]
    return features

# Iterate over each EDF and annotation file pair
for i in range(1, 155):  # Adjust range for files from SN001 to SN154
    if i == 138:
        continue  # Skip SN138
    
    # Format the index to include leading zeros
    file_index = f'{i:03}'  # Ensures the file index has three digits, e.g., 001, 002, ..., 154
    edf_file = os.path.join(edf_dir, f'SN{file_index}.edf')
    annotation_file = os.path.join(annotation_dir, f'SN{file_index}_sleepscoring.txt')


    
    # Load EDF file
    raw = mne.io.read_raw_edf(edf_file, preload=True)
    raw.crop(tmin=150, tmax=raw.times[-1] - 150)  # Crop 150s from start and end
    raw.filter(0.5, 30, fir_design='firwin')  # Bandpass filter
    raw.resample(100)  # Resample

    # Load and process annotation file
    annotations_df = pd.read_csv(annotation_file)
    annotations_df.columns = ["Date", "Time", "Recording onset", "Duration", "Annotation", "Linked channel"]
    annotations_df['Recording onset'] = pd.to_numeric(annotations_df['Recording onset'], errors='coerce')
    annotations_df['Duration'] = pd.to_numeric(annotations_df['Duration'], errors='coerce')
    
    # Filter annotations to fit within the cropped data range and adjust onset
    total_duration = raw.times[-1] - 150 * 2
    filtered_annotations_df = annotations_df[
        (annotations_df['Recording onset'] >= 150) &
        (annotations_df['Recording onset'] <= total_duration)
    ].copy()
    filtered_annotations_df['Recording onset'] -= 150
    
    # Set annotations
    annotations = mne.Annotations(
        onset=filtered_annotations_df['Recording onset'].values,
        duration=filtered_annotations_df['Duration'].values,
        description=filtered_annotations_df['Annotation'].values
    )
    raw.set_annotations(annotations)
    
    # Create events and epochs
    events, event_id = mne.events_from_annotations(raw)
    epochs = mne.Epochs(raw, events, event_id, tmin=0, tmax=30, baseline=None, preload=True)
    
    # Encode labels
    labels = epochs.events[:, -1]
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    
    # Wavelet feature extraction
    eeg_data = epochs.get_data()  # Shape: (n_epochs, n_channels, n_timepoints)
    wavelet_features = []
    for epoch in eeg_data:
        for channel_data in epoch:
            features = compute_wavelet_features(channel_data)
            wavelet_features.append(features)
    wavelet_features = np.array(wavelet_features)
    wavelet_features = wavelet_features.reshape(len(labels), -1)  # Reshape for ML input

    # Append to the main lists
    all_wavelet_features.extend(wavelet_features)
    all_labels.extend(labels)

# Save preprocessed data for reuse
np.save('wavelet_features.npy', all_wavelet_features)
np.save('wavelet_labels.npy', all_labels)
print("Data saved successfully.")


Extracting EDF parameters from c:\Users\lewin\OneDrive\Desktop\College Semesters\Sleep Stage Classification\physionet.org\files\hmc-sleep-staging\1.0.0\recordings\SN001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6566399  =      0.000 ... 25649.996 secs...


C:\Users\lewin\AppData\Local\Temp\ipykernel_25752\3539974303.py:40: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=True)
C:\Users\lewin\AppData\Local\Temp\ipykernel_25752\3539974303.py:40: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1691 samples (6.605 s)

Used Annotations descriptions: [np.str_(' Sleep stage N1'), np.str_(' Sleep stage N2'), np.str_(' Sleep stage N3'), np.str_(' Sleep stage R'), np.str_(' Sleep stage W')]
Not setting metadata
830 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 830 events and 3001 original time points ...
0 bad epochs dropped
Extracting EDF parameters from c:\Users\lewin\OneDrive\Deskt

C:\Users\lewin\AppData\Local\Temp\ipykernel_25752\3539974303.py:40: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=True)
C:\Users\lewin\AppData\Local\Temp\ipykernel_25752\3539974303.py:40: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1691 samples (6.605 s)

Used Annotations descriptions: [np.str_(' Sleep stage N1'), np.str_(' Sleep stage N2'), np.str_(' Sleep stage N3'), np.str_(' Sleep stage R'), np.str_(' Sleep stage W')]
Not setting metadata
832 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 832 events and 3001 original time points ...
0 bad epochs dropped
Extracting EDF parameters from c:\Users\lewin\OneDrive\Deskt

...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1691 samples (6.605 s)

Used Annotations descriptions: [np.str_(' Sleep stage N1'), np.str_(' Sleep stage N2'), np.str_(' Sleep stage N3'), np.str_(' Sleep stage R'), np.str_(' Sleep stage W')]
Not setting metadata
975 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 975 events and 3001 original time points ...
0 bad epochs dropped
Extracting EDF parameters from c:\Users\lewin\OneDrive\Deskt

C:\Users\lewin\AppData\Local\Temp\ipykernel_25752\3539974303.py:40: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=True)
C:\Users\lewin\AppData\Local\Temp\ipykernel_25752\3539974303.py:40: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1691 samples (6.605 s)

Used Annotations descriptions: [np.str_(' Sleep stage N1'), np.str_(' Sleep stage N2'), np.str_(' Sleep stage N3'), np.str_(' Sleep stage R'), np.str_(' Sleep stage W')]
Not setting metadata
878 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 878 events and 3001 original time points ...
0 bad epochs dropped
Extracting EDF parameters from c:\Users\lewin\OneDrive\Deskt

C:\Users\lewin\AppData\Local\Temp\ipykernel_25752\3539974303.py:40: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=True)
C:\Users\lewin\AppData\Local\Temp\ipykernel_25752\3539974303.py:40: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1691 samples (6.605 s)

Used Annotations descriptions: [np.str_(' Sleep stage N1'), np.str_(' Sleep stage N2'), np.str_(' Sleep stage N3'), np.str_(' Sleep stage R'), np.str_(' Sleep stage W')]
Not setting metadata
979 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 979 events and 3001 original time points ...
0 bad epochs dropped
Data saved successfully.


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Load pre-saved data
all_wavelet_features = np.load('wavelet_features.npy')
all_labels = np.load('wavelet_labels.npy')

print("Data loaded successfully.")

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(all_wavelet_features, all_labels, test_size=0.2, random_state=42)
# Initialize and train the classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
print("Model training completed.")

# Make predictions
y_pred = clf.predict(X_test)

# Display accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
classification_report_dict = classification_report(y_test, y_pred, output_dict=True)
confusion_matrix = ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, display_labels=np.unique(all_labels)
).confusion_matrix
cv_scores_rf = cross_val_score(clf, all_wavelet_features, all_labels, cv=5)

# Save results
np.save('accuracy.npy', np.array([accuracy]))
np.save('classification_report.npy', np.array(classification_report_dict, dtype=object))
np.save('cv_scores.npy', np.array(cv_scores_rf))
np.save('confusion_matrix.npy', confusion_matrix)

print("Model outputs saved successfully.")
